# Initial screening

This notebook supports reviewer-sharded screening with human Y/N/M decisions. The LLM (Gemini 2.5 Flash) only provides a short summary and hints for the inclusion criteria.

Workflow:
- Configure env and paths
- Load input CSV (existing schema) depending on first or second initial screening
- Reproducible random sharding across reviewers
- For your assigned papers: review each, see LLM summary/hints, choose Y/N/M (+ optional notes)
- Append decisions to per-reviewer CSV; export all M to a `maybes.csv` at the end


In [1]:
import os
import json
from datetime import datetime
from pathlib import Path

import pandas as pd

# Config
REVIEWERS = ["Olav", "Ulrik", "Trine"]
CURRENT_REVIEWER = os.environ.get("CURRENT_REVIEWER", "Olav")  # or set manually later
SEED = int(os.environ.get("SCREENING_SEED", 42))

# Paths
PROJECT_ROOT = Path(".")
INPUT_CSV = PROJECT_ROOT / "articles_with_gpt_response.csv"
# OUT_DIR = PROJECT_ROOT / "outputs" # For first initial screening
# ASSIGNMENTS_CSV = OUT_DIR / "assignments.csv" # First initial screening

OUT_DIR = PROJECT_ROOT / "outputs" / "initial_screening_2" # For second initial screening
ASSIGNMENTS_CSV = OUT_DIR / "assignments2.csv" # Second initial screening

OUT_DIR.mkdir(parents=True, exist_ok=True)

print({
    "current_reviewer": CURRENT_REVIEWER,
    "input_csv": str(INPUT_CSV),
    "out_dir": str(OUT_DIR),
})


{'current_reviewer': 'Olav', 'input_csv': 'articles_with_gpt_response.csv', 'out_dir': 'outputs\\initial_screening_2'}


In [2]:
# Load enhanced dataset and assignments
df = pd.read_csv(INPUT_CSV)
assignments = pd.read_csv(ASSIGNMENTS_CSV)
print("Loaded:", len(df), "papers; assignments:", len(assignments))

Loaded: 1203 papers; assignments: 1203


We re-specify the inclusion criteria:

In [3]:
# Review-only notebook: criteria constants

CRITERIA_TEXT = {
    "I1": "The article includes an empirical Machine Learning application (not pure theory). It goes beyond simple linear models.",
    "I2": "The article is about government/sovereign bonds and/or the yield curve",
    "I3": "The article is either about forecasting or estimating",
}

CRITERIA_TEXT


{'I1': 'The article includes an empirical Machine Learning application (not pure theory). It goes beyond simple linear models.',
 'I2': 'The article is about government/sovereign bonds and/or the yield curve',
 'I3': 'The article is either about forecasting or estimating'}

We create a simple screening widget to facilitate for screening, and run through the assigned papers separatel:

In [4]:
# Simple screening app (ipywidgets)

import ipywidgets as w
from IPython.display import display, clear_output

reviewer_dropdown = w.Dropdown(options=["Olav", "Ulrik", "Trine"], value=CURRENT_REVIEWER, description='Reviewer:')
count_label = w.HTML()

title_html = w.HTML()
meta_html = w.HTML()
current_label_html = w.HTML()
abstract_area = w.HTML(layout=w.Layout(width='100%', max_height='400px', overflow_y='auto', border='1px solid #ddd', padding='10px'))
summary_area = w.HTML(layout=w.Layout(width='100%', max_height='200px', overflow_y='auto', border='1px solid #ddd', padding='10px'))
hints_html = w.HTML()
criteria_html = w.HTML(
    "<b>Criteria</b><br>" + "<br>".join([f"{k}: {v}" for k,v in CRITERIA_TEXT.items()])
)

prev_btn = w.Button(description='Prev', button_style='')
next_btn = w.Button(description='Next', button_style='')
y_btn = w.Button(description='Yes (Y)', button_style='success')
n_btn = w.Button(description='No (N)', button_style='danger')
m_btn = w.Button(description='Maybe (M)', button_style='warning')
review_btn = w.Button(description='Review (R)', button_style='info')
notes_txt = w.Textarea(placeholder='Optional notes...', layout=w.Layout(width='100%', height='60px'))
status_html = w.HTML()

nav_box = w.HBox([prev_btn, next_btn, y_btn, n_btn, m_btn, review_btn])
ui = w.VBox([
    reviewer_dropdown,
    count_label,
    current_label_html,
    title_html,
    meta_html,
    abstract_area,
    summary_area,
    hints_html,
    criteria_html,
    notes_txt,
    nav_box,
    status_html,
])

# State
idx = 0
current_reviewer = reviewer_dropdown.value
reviewer_out_csv = OUT_DIR / f"{current_reviewer}_decisions.csv"

# Load existing decisions
if reviewer_out_csv.exists():
    decided_df = pd.read_csv(reviewer_out_csv)
    decisions_dict = {str(row['paper_id']): row.to_dict() for _, row in decided_df.iterrows()}
else:
    decided_df = pd.DataFrame()
    decisions_dict = {}

# Load all assigned papers (not just remaining)
assigned_df = df.merge(assignments[["paper_id", "reviewer"]], on="paper_id", how="left")
assigned_df = assigned_df[assigned_df["reviewer"] == current_reviewer].reset_index(drop=True)
queue_df = assigned_df.copy()


def update_count():
    # Count papers without any decision (Y/N/M/Review)
    remaining = sum(1 for _, r in queue_df.iterrows() if str(r['paper_id']) not in decisions_dict)
    count_label.value = f"<b>{current_reviewer}</b>: {remaining} remaining of {len(queue_df)} | Viewing: {idx + 1}/{len(queue_df)}"


def refresh_state():
    global current_reviewer, reviewer_out_csv, decided_df, decisions_dict, assigned_df, queue_df, idx
    current_reviewer = reviewer_dropdown.value
    reviewer_out_csv = OUT_DIR / f"{current_reviewer}_decisions.csv"
    if reviewer_out_csv.exists():
        decided_df = pd.read_csv(reviewer_out_csv)
        decisions_dict = {str(row['paper_id']): row.to_dict() for _, row in decided_df.iterrows()}
    else:
        decided_df = pd.DataFrame()
        decisions_dict = {}
    assigned_df = df.merge(assignments[["paper_id", "reviewer"]], on="paper_id", how="left")
    assigned_df = assigned_df[assigned_df["reviewer"] == current_reviewer].reset_index(drop=True)
    queue_df = assigned_df.copy()
    idx = 0
    show(idx)


def show(i: int):
    if len(queue_df) == 0:
        title_html.value = "<b>No papers assigned or all done.</b>"
        meta_html.value = ""
        current_label_html.value = ""
        abstract_area.value = ""
        summary_area.value = ""
        hints_html.value = ""
        update_count()
        return
    r = queue_df.iloc[i]
    paper_id = str(r.get('paper_id'))
    
    # Show current label if exists
    current_decision = decisions_dict.get(paper_id)
    if current_decision:
        decision_label = current_decision.get('decision', 'Unknown')
        current_label_html.value = f'<div style="background-color: #e3f2fd; padding: 5px; border-radius: 4px;"><b>Current Label:</b> {decision_label}</div>'
        notes_txt.value = str(current_decision.get('notes', ''))
    else:
        current_label_html.value = '<div style="background-color: #fff3cd; padding: 5px; border-radius: 4px;"><b>Current Label:</b> None</div>'
        notes_txt.value = ""
    
    title_html.value = f"<h3>{r.get('title','')}</h3>"
    meta_html.value = f"Year: {r.get('year','')} | DOI: {r.get('doi','')} | DB: {r.get('database','')}"
    abstract_area.value = f'<div style="font-family: -apple-system, BlinkMacSystemFont, \'Segoe UI\', Roboto, Arial, sans-serif; font-size: 12px; line-height: 1.6; color: #000;"><b>Abstract:</b><br>{str(r.get("abstract",""))}</div>'
    summary_area.value = f'<div style="font-family: -apple-system, BlinkMacSystemFont, \'Segoe UI\', Roboto, Arial, sans-serif; font-size: 12px; line-height: 1.6; color: #000;"><b>LLM Summary:</b><br>{r.get("llm_summary","")}</div>'
    
    # Use large colored checkmark/cross emojis for hints
    def format_hint(val):
        if val == True:
            return '<span style="color: green; font-size: 28px; font-weight: bold;">✓</span>'
        elif val == False:
            return '<span style="color: red; font-size: 28px; font-weight: bold;">✗</span>'
        else:
            return '<span style="color: gray; font-size: 28px;">?</span>'
    
    hints_html.value = f'''
    <div style="margin: 10px 0;">
        <b style="font-size: 16px;">Hints:</b><br>
        <div style="margin-top: 8px; display: flex; align-items: center;">
            <span style="margin-right: 30px; display: inline-flex; align-items: center;"><b>I1:</b> {format_hint(r.get('i1_hint'))}</span>
            <span style="margin-right: 30px; display: inline-flex; align-items: center;"><b>I2:</b> {format_hint(r.get('i2_hint'))}</span>
            <span style="margin-right: 30px; display: inline-flex; align-items: center;"><b>I3:</b> {format_hint(r.get('i3_hint'))}</span>
        </div>
    </div>
    '''
    update_count()


refresh_state()


def save_decision(decision: str):
    if len(queue_df) == 0:
        return
    r = queue_df.iloc[idx]
    paper_id = str(r.get("paper_id"))
    
    new_row = {
        **{c: r.get(c) for c in df.columns if c in r},
        "paper_id": paper_id,
        "reviewer": current_reviewer,
        "decision": decision,
        "notes": notes_txt.value,
        "i1_hint": r.get('i1_hint'),
        "i2_hint": r.get('i2_hint'),
        "i3_hint": r.get('i3_hint'),
        "llm_summary": r.get('llm_summary'),
        "timestamp": datetime.utcnow().isoformat(),
    }
    
    # Update or add decision
    if reviewer_out_csv.exists():
        existing_df = pd.read_csv(reviewer_out_csv)
        # Remove old entry if exists
        existing_df = existing_df[existing_df['paper_id'].astype(str) != paper_id]
        # Add new entry
        updated_df = pd.concat([existing_df, pd.DataFrame([new_row])], ignore_index=True)
        updated_df.to_csv(reviewer_out_csv, index=False)
    else:
        pd.DataFrame([new_row]).to_csv(reviewer_out_csv, index=False)
    
    # Update in-memory decisions dict
    decisions_dict[paper_id] = new_row
    
    status_html.value = f"Saved decision {decision} for {paper_id}"
    update_count()


def on_prev(_):
    global idx
    if idx > 0:
        idx -= 1
        show(idx)

def on_next(_):
    global idx
    if idx < len(queue_df) - 1:
        idx += 1
        show(idx)

def on_y(_):
    save_decision('Y')
    on_next(_)

def on_n(_):
    save_decision('N')
    on_next(_)

def on_m(_):
    save_decision('M')
    on_next(_)

def on_review(_):
    save_decision('Review')
    on_next(_)

prev_btn.on_click(on_prev)
next_btn.on_click(on_next)
y_btn.on_click(on_y)
n_btn.on_click(on_n)
m_btn.on_click(on_m)
review_btn.on_click(on_review)
reviewer_dropdown.observe(lambda change: refresh_state() if change['name']=='value' else None, names='value')

display(ui)

In [8]:
# Export maybes across all reviewers

all_decisions = []
for name in REVIEWERS:
    p = OUT_DIR / f"{name}_decisions.csv"
    if p.exists():
        all_decisions.append(pd.read_csv(p))

if all_decisions:
    merged = pd.concat(all_decisions, ignore_index=True)
    maybes = merged[merged["decision"] == "M"].copy()
    maybes_out = OUT_DIR / "maybes.csv"
    maybes.to_csv(maybes_out, index=False)
    print("Maybes saved:", maybes_out, "| count:", len(maybes))
else:
    print("No decision files found yet.")


Maybes saved: outputs/maybes.csv | count: 128


## Maybe screening


In [9]:
# Maybe Resolution App - Screen through papers marked as "Maybe"
print("🔍 Maybe Resolution Screening Interface")
print("=" * 50)

# Load maybes.csv if it exists
maybes_csv = OUT_DIR / "maybes.csv"
resolved_maybes_csv = OUT_DIR / "resolved_maybes.csv"

if not maybes_csv.exists():
    print("❌ No maybes.csv file found. Run the 'Export maybes' cell first.")
else:
    maybes_df = pd.read_csv(maybes_csv)
    print(f"📋 Loaded {len(maybes_df)} papers marked as 'Maybe'")
    
    # Create a separate screening interface for maybes
    maybe_reviewer_dropdown = w.Dropdown(options=["Olav", "Ulrik", "Trine"], value=CURRENT_REVIEWER, description='Reviewer:')
    maybe_count_label = w.HTML()
    
    maybe_title_html = w.HTML()
    maybe_meta_html = w.HTML()
    maybe_current_label_html = w.HTML()
    maybe_abstract_area = w.HTML(layout=w.Layout(width='100%', max_height='400px', overflow_y='auto', border='1px solid #ddd', padding='10px'))
    maybe_summary_area = w.HTML(layout=w.Layout(width='100%', max_height='200px', overflow_y='auto', border='1px solid #ddd', padding='10px'))
    maybe_hints_html = w.HTML()
    maybe_criteria_html = w.HTML(
        "<b>Criteria</b><br>" + "<br>".join([f"{k}: {v}" for k,v in CRITERIA_TEXT.items()])
    )
    
    maybe_prev_btn = w.Button(description='Prev', button_style='')
    maybe_next_btn = w.Button(description='Next', button_style='')
    maybe_y_btn = w.Button(description='Yes (Y)', button_style='success')
    maybe_n_btn = w.Button(description='No (N)', button_style='danger')
    maybe_notes_txt = w.Textarea(placeholder='Optional notes...', layout=w.Layout(width='100%', height='60px'))
    maybe_status_html = w.HTML()
    
    maybe_nav_box = w.HBox([maybe_prev_btn, maybe_next_btn, maybe_y_btn, maybe_n_btn])
    maybe_ui = w.VBox([
        w.HTML("<h3>🔍 Maybe Resolution Screening</h3>"),
        maybe_reviewer_dropdown,
        maybe_count_label,
        maybe_current_label_html,
        maybe_title_html,
        maybe_meta_html,
        maybe_abstract_area,
        maybe_summary_area,
        maybe_hints_html,
        maybe_criteria_html,
        maybe_notes_txt,
        maybe_nav_box,
        maybe_status_html,
    ])
    
    # State for maybe screening
    maybe_idx = 0
    maybe_current_reviewer = maybe_reviewer_dropdown.value
    
    # Load existing resolved maybes
    if resolved_maybes_csv.exists():
        resolved_df = pd.read_csv(resolved_maybes_csv)
        resolved_dict = {str(row['paper_id']): row.to_dict() for _, row in resolved_df.iterrows()}
    else:
        resolved_df = pd.DataFrame()
        resolved_dict = {}
    
    def update_maybe_count():
        remaining = sum(1 for _, r in maybes_df.iterrows() if str(r['paper_id']) not in resolved_dict)
        maybe_count_label.value = f"<b>{maybe_current_reviewer}</b>: {remaining} remaining of {len(maybes_df)} | Viewing: {maybe_idx + 1}/{len(maybes_df)}"
    
    def show_maybe(i: int):
        if len(maybes_df) == 0:
            maybe_title_html.value = "<b>No maybe papers to review.</b>"
            maybe_meta_html.value = ""
            maybe_current_label_html.value = ""
            maybe_abstract_area.value = ""
            maybe_summary_area.value = ""
            maybe_hints_html.value = ""
            update_maybe_count()
            return
        
        r = maybes_df.iloc[i]
        paper_id = str(r.get('paper_id'))
        
        # Show current resolution if exists
        current_resolution = resolved_dict.get(paper_id)
        if current_resolution:
            decision_label = current_resolution.get('decision', 'Unknown')
            maybe_current_label_html.value = f'<div style="background-color: #e3f2fd; padding: 5px; border-radius: 4px;"><b>Current Resolution:</b> {decision_label}</div>'
            maybe_notes_txt.value = str(current_resolution.get('notes', ''))
        else:
            maybe_current_label_html.value = '<div style="background-color: #fff3cd; padding: 5px; border-radius: 4px;"><b>Current Resolution:</b> None (was Maybe)</div>'
            maybe_notes_txt.value = ""
        
        maybe_title_html.value = f"<h3>{r.get('title','')}</h3>"
        maybe_meta_html.value = f"Year: {r.get('year','')} | DOI: {r.get('doi','')} | DB: {r.get('database','')} | Original Reviewer: {r.get('reviewer','')}"
        maybe_abstract_area.value = f'<div style="font-family: -apple-system, BlinkMacSystemFont, \'Segoe UI\', Roboto, Arial, sans-serif; font-size: 12px; line-height: 1.6; color: #000;"><b>Abstract:</b><br>{str(r.get("abstract",""))}</div>'
        maybe_summary_area.value = f'<div style="font-family: -apple-system, BlinkMacSystemFont, \'Segoe UI\', Roboto, Arial, sans-serif; font-size: 12px; line-height: 1.6; color: #000;"><b>LLM Summary:</b><br>{r.get("llm_summary","")}</div>'
        
        # Use large colored checkmark/cross emojis for hints
        def format_hint(val):
            if val == True:
                return '<span style="color: green; font-size: 28px; font-weight: bold;">✓</span>'
            elif val == False:
                return '<span style="color: red; font-size: 28px; font-weight: bold;">✗</span>'
            else:
                return '<span style="color: gray; font-size: 28px;">?</span>'
        
        maybe_hints_html.value = f'''
        <div style="margin: 10px 0;">
            <b style="font-size: 16px;">Hints:</b><br>
            <div style="margin-top: 8px; display: flex; align-items: center;">
                <span style="margin-right: 30px; display: inline-flex; align-items: center;"><b>I1:</b> {format_hint(r.get('i1_hint'))}</span>
                <span style="margin-right: 30px; display: inline-flex; align-items: center;"><b>I2:</b> {format_hint(r.get('i2_hint'))}</span>
                <span style="margin-right: 30px; display: inline-flex; align-items: center;"><b>I3:</b> {format_hint(r.get('i3_hint'))}</span>
            </div>
        </div>
        '''
        update_maybe_count()
    
    def save_maybe_decision(decision: str):
        if len(maybes_df) == 0:
            return
        
        r = maybes_df.iloc[maybe_idx]
        paper_id = str(r.get("paper_id"))
        
        new_row = {
            **{c: r.get(c) for c in maybes_df.columns if c in r},
            "paper_id": paper_id,
            "resolver": maybe_current_reviewer,
            "decision": decision,
            "notes": maybe_notes_txt.value,
            "original_reviewer": r.get('reviewer'),
            "original_decision": "M",
            "timestamp": datetime.utcnow().isoformat(),
        }
        
        # Update or add resolution
        if resolved_maybes_csv.exists():
            existing_df = pd.read_csv(resolved_maybes_csv)
            # Remove old entry if exists
            existing_df = existing_df[existing_df['paper_id'].astype(str) != paper_id]
            # Add new entry
            updated_df = pd.concat([existing_df, pd.DataFrame([new_row])], ignore_index=True)
            updated_df.to_csv(resolved_maybes_csv, index=False)
        else:
            pd.DataFrame([new_row]).to_csv(resolved_maybes_csv, index=False)
        
        # Update in-memory resolutions dict
        resolved_dict[paper_id] = new_row
        
        maybe_status_html.value = f"Saved resolution {decision} for {paper_id}"
        update_maybe_count()
    
    def on_maybe_prev(_):
        global maybe_idx
        if maybe_idx > 0:
            maybe_idx -= 1
            show_maybe(maybe_idx)
    
    def on_maybe_next(_):
        global maybe_idx
        if maybe_idx < len(maybes_df) - 1:
            maybe_idx += 1
            show_maybe(maybe_idx)
    
    def on_maybe_y(_):
        save_maybe_decision('Y')
        on_maybe_next(_)
    
    def on_maybe_n(_):
        save_maybe_decision('N')
        on_maybe_next(_)
    
    def refresh_maybe_state():
        global maybe_current_reviewer, maybe_idx
        maybe_current_reviewer = maybe_reviewer_dropdown.value
        maybe_idx = 0
        show_maybe(maybe_idx)
    
    # Connect event handlers
    maybe_prev_btn.on_click(on_maybe_prev)
    maybe_next_btn.on_click(on_maybe_next)
    maybe_y_btn.on_click(on_maybe_y)
    maybe_n_btn.on_click(on_maybe_n)
    maybe_reviewer_dropdown.observe(lambda change: refresh_maybe_state() if change['name']=='value' else None, names='value')
    
    # Initialize
    show_maybe(maybe_idx)
    display(maybe_ui)


🔍 Maybe Resolution Screening Interface
📋 Loaded 128 papers marked as 'Maybe'
